In [ ]:
# https://www.youtube.com/watch?v=xOccYkgRV4Q&ab_channel=CampusX

In [132]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

###
1. simple imputer null values of age(fill it with mean) , embarked (fill with most frequent value)
2. OHE on sex and embarked
3. scale numeric features
4. select K best features  - feature selection
5. feed to decision tree
6. make pipeline of above transformations

In [82]:
x = pd.read_csv('titanic.csv')
x.drop(columns = ['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [83]:
x.sample(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
398,0,3,male,30.0,1,0,16.1000,S
887,0,2,male,NaN,0,0,0.0000,S
747,0,3,male,16.0,0,0,7.7750,S
72,1,1,female,NaN,1,0,82.1708,C
152,0,1,male,NaN,0,0,52.0000,S


In [84]:
x_train,x_test,y_train,y_test = train_test_split(x.drop(columns=['Survived']),x['Survived'],test_size=0.1,random_state=42)

In [85]:
x_train.shape,x_test.shape,y_train.shape

((801, 7), (90, 7), (801,))

In [86]:
x_train.sample(5)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
295,2,male,35.0,0,0,26.0000,S
866,3,male,NaN,0,0,6.8583,Q
49,1,female,17.0,1,0,108.9000,C
559,3,female,21.0,1,0,9.8250,S
338,3,male,36.0,1,1,24.1500,S


In [87]:
# Simple imputation on age and embarked
tf1 = ColumnTransformer(transformers=[('impute_age',SimpleImputer(strategy='mean'),[2]),
                                        ('impute_emb',SimpleImputer(strategy='most_frequent'),[6])],remainder='passthrough')

In [88]:
#OHE on sex and embarked
tf2 = ColumnTransformer([('ohe_sex_embarked',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6])]
                                       ,remainder='passthrough')

In [94]:
#scaling of numeric columns
tf3 = ColumnTransformer(transformers=[('scalex',MinMaxScaler(),slice(0,10))]
                        )

In [90]:
#select k best features
tf4 = SelectKBest(score_func=chi2,k=8)

In [91]:
# feed to decision tree
tf5 = DecisionTreeClassifier()

### Create Pipeline

In [95]:
pipe = Pipeline([('tf1',tf1),
                 ('tf2',tf2),
                 ('tf3',tf3),
                 ('tf4',tf4),
                 ('tf5',tf5)])

In [96]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('tf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_emb',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('tf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('tf3',
                 ColumnTransformer(transformers=[('scalex', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('tf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000002117B214790>)),
                ('tf5', DecisionTreeClassifier())])

In [36]:
from sklearn import set_config
set_config(display='diagram')

In [97]:
y_pred = pipe.predict(x_test)

In [99]:
accuracy_score(y_test,y_pred)

0.5666666666666667

### Explore the pipeline

In [102]:
pipe.named_steps

{'tf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_emb',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'tf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [1, 6])]),
 'tf3': ColumnTransformer(transformers=[('scalex', MinMaxScaler(), slice(0, 10, None))]),
 'tf4': SelectKBest(k=8, score_func=<function chi2 at 0x000002117B214790>),
 'tf5': DecisionTreeClassifier()}

In [107]:
pipe.named_steps['tf1'].transformers_

[('impute_age', SimpleImputer(), [2]),
 ('impute_emb', SimpleImputer(strategy='most_frequent'), [6]),
 ('remainder', 'passthrough', [0, 1, 3, 4, 5])]

In [111]:
pipe.named_steps['tf1'].transformers_[0][1]

SimpleImputer()

In [112]:
pipe.named_steps['tf1'].transformers_[0][1].statistics_ #mean age as strategy used was mean

array([29.30201863])

In [113]:
pipe.named_steps['tf1'].transformers_[1][1].statistics_  #most frequent embarked value was 'S''

array(['S'], dtype=object)

In [114]:
pipe.named_steps['tf2']

ColumnTransformer(remainder='passthrough',
                  transformers=[('ohe_sex_embarked',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse=False),
                                 [1, 6])])

In [115]:
pipe.named_steps['tf2'].transformers_

[('ohe_sex_embarked',
  OneHotEncoder(handle_unknown='ignore', sparse=False),
  [1, 6]),
 ('remainder', 'passthrough', [0, 2, 3, 4, 5])]

In [117]:
pipe.named_steps['tf2'].transformers_[0][1]

OneHotEncoder(handle_unknown='ignore', sparse=False)

In [119]:
pipe.named_steps['tf2'].transformers_[0][1].categories_

[array(['C', 'Q', 'S'], dtype=object),
 array([0.0, 4.0125, 5.0, 6.2375, 6.4375, 6.45, 6.4958, 6.75, 6.8583, 6.95,
        6.975, 7.0458, 7.05, 7.0542, 7.125, 7.1417, 7.225, 7.2292, 7.25,
        7.3125, 7.4958, 7.5208, 7.55, 7.6292, 7.65, 7.7292, 7.7333, 7.7375,
        7.7417, 7.75, 7.775, 7.7875, 7.7958, 7.8, 7.8292, 7.8542, 7.875,
        7.8792, 7.8875, 7.8958, 7.925, 8.0292, 8.05, 8.1375, 8.1583, 8.3,
        8.3625, 8.4042, 8.4333, 8.4583, 8.5167, 8.6542, 8.6625, 8.6833,
        8.7125, 8.85, 9.0, 9.2167, 9.225, 9.35, 9.475, 9.4833, 9.5, 9.5875,
        9.825, 9.8375, 9.8417, 9.8458, 10.1708, 10.4625, 10.5, 10.5167,
        11.1333, 11.2417, 11.5, 12.0, 12.275, 12.2875, 12.35, 12.475,
        12.525, 12.65, 12.875, 13.0, 13.4167, 13.5, 13.7917, 13.8583,
        13.8625, 14.0, 14.1083, 14.4, 14.4542, 14.4583, 14.5, 15.0,
        15.0458, 15.05, 15.1, 15.2458, 15.5, 15.55, 15.7417, 15.75, 15.85,
        15.9, 16.0, 16.1, 16.7, 17.4, 17.8, 18.0, 18.75, 18.7875, 19.2583,
        19.

In [120]:
pipe.named_steps['tf2'].transformers_[0][1].n_features_in_

2

### Cross validation using pipelines

In [128]:
cross_val = cross_val_score(pipe,x_train,y_train,cv=5,scoring='accuracy')

In [131]:
cross_val.mean() #mean accuracy after splitting and training 5 times

0.6441614906832298

### GridSearch and Hyper-parameter tuning for decision tree classifier

In [133]:
params = {
            'tf5__max_depth':[1,2,3,4,5,None]
            }

In [134]:
grid = GridSearchCV(pipe,params,cv=5,scoring='accuracy')
grid.fit(x_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_emb',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('tf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         [1,
                                                                          6])])),
                                       ('tf3',
                                        ColumnTransformer(transformers=[('scalex',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('tf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x000002117B214790>)),
                                       ('tf5', DecisionTreeClassifier())]),
             param_grid={'tf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [135]:
grid.best_score_

0.6441614906832298

In [136]:
grid.best_params_

{'tf5__max_depth': 1}

### Exporting the pipeline

In [137]:
import pickle

In [138]:
pickle.dump(pipe,open('pipe.pkl','wb'))